In [ ]:
# tripadvisor review webscraper
# author: Mohamed Wishan
# date: 09.92.2021
# © 2021 Mohamed Wishan, CC BY 4.0

# install required libraries
!pip install mysql-connector-python
!pip install requests-html

In [ ]:
# import required libraries
import time
import datetime
import mysql.connector
from requests_html import HTMLSession
from IPython.display import clear_output
from mysql.connector.constants import ClientFlag
from numpy import random
from time import sleep

In [ ]:
# set up mysql SSL connection: make sure to insert your details
config = {
    'user': 'yourusername',
    'password': 'yourpassword',
    'host': 'youripaddress',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem'
}
try:
    # add existing database to config dict
    config['database'] = 'webscraper'
    cnxn = mysql.connector.connect(**config)
    cursor = cnxn.cursor()
    print("Database connection successful")
except Exception as e:
    print("Could not connect to the database")
    print(e)

In [ ]:
hotel_list = []
try:
    # query list of nonscraped hotels
    cursor.execute("SELECT * FROM hotels WHERE hotel_scraped=0")
    #cursor.execute("SELECT * FROM hotels WHERE id=62")
    hotel_list = cursor.fetchall()
    if bool(hotel_list):
        print("Hotels ready to scrape")
    else:
        print("All hotels scraped")
except Exception as e:
    print("Could not run query")
    print(e)

In [ ]:
# rating dictionary to translate element class name into rating values
rating_dict = {
    50 : "bubble_50",
    45 : "bubble_45" ,
    40 : "bubble_40",
    35 : "bubble_35",
    30 : "bubble_30",
    25 : "bubble_25",
    20 : "bubble_20",
    15 : "bubble_15",
    10 : "bubble_10",
     5 : "bubble_05",
     0 : "bubble_00",
}

#FLAGS FOR ERRORS
error_hotel_rows = []

In [ ]:
# LOOP THROUGH ALL HOTELS
for i in hotel_list:
    clear_output(wait=True)
    try:
        session = HTMLSession()
        r = session.get(i[3])
    except Exception as e:
        error_hotel_rows.append(e)
        print("Could not retrieve hotel from tripadvisor")
        print(e)

    sleeptime = random.uniform(4, 9)
    print("sleeping for:", sleeptime, "seconds")
    sleep(sleeptime)
    print("sleeping is over")
    
    print("SCRAPING " + i[2])
    print("ID: "+str(i[0]))
    print("URL: " + i[3])
    page_num = 0
    
    main_page = r.html.url
    # LOOP THROUGH HOTEL PAGES
    for html in r.html:
        review_num = 0
        print(html)
        
        try:
            r = session.get(html.url)
            r_username  = r.html.find('.ui_header_link')  # raw username
            r_review    = r.html.find('.IRsGHoPm')        # raw review
            r_date      = r.html.find('._34Xs-BQm')       # raw date substring 14:
            r_rating    = r.html.find('.nf9vGX55')        # raw ratings html element
            review_num  = len(r_username)                 # number of reviews on current page
            #print(review_num)
        except Exception as e:
            error_hotel_rows.append(e)
            print("Could not retrieve hotel from tripadvisor")
            print(e)
            
        if page_num==1 and html.url[:-10]==main_page:
            break
        elif r_username==[]:
            print("NO REVIEWS ON PAGE ",page_num)
            break
        
        
        
        # LOOP THROUGH REVIEWS ON THIS PAGE
        review_list = []
        for x in range(review_num):         
            # CLEAN DATA
            hotel_id = i[0]
            username = r_username[x].text
            review = r_review[x].text
            # print(review)
            if    rating_dict[50] in r_rating[x].html : rate = 50
            elif  rating_dict[45] in r_rating[x].html : rate = 45
            elif  rating_dict[40] in r_rating[x].html : rate = 40
            elif  rating_dict[35] in r_rating[x].html : rate = 35
            elif  rating_dict[30] in r_rating[x].html : rate = 30
            elif  rating_dict[25] in r_rating[x].html : rate = 25
            elif  rating_dict[20] in r_rating[x].html : rate = 20
            elif  rating_dict[15] in r_rating[x].html : rate = 15
            elif  rating_dict[10] in r_rating[x].html : rate = 10
            elif  rating_dict[5]  in r_rating[x].html : rate = 5
            else: rate = 0
            date = datetime.datetime.strptime("December 2005", '%B %Y')   
            try:
                date = datetime.datetime.strptime(r_date[x].text[14:], '%B %Y')
            except Exception as e:
                print("datetime error")
                error_hotel_rows.append(e)
                print(e)
            
            review_tuple = (hotel_id, username, rate, review, date)
            review_list.append(review_tuple)
            
            #LOOKING FOR ERROR
            
        # EXECUTE DATABASE CMD TO INSERT ALL HOTELS ON PAGE
        try:
            insert_review = ("INSERT INTO review (hotel_id, username, rating, review, date) VALUES (%s, %s, %s, %s, %s )")
            cursor.executemany(insert_review, review_list)
            cnxn.commit()
            print(">> PAGE "+str(page_num)+" SUCCESSFULLY SCRAPED")
        except Exception as e:
            print("SQL ERROR: COULD NOT INSERT INTO DB")
            error_hotel_rows.append(e)
            print(e)
            
 
        page_num+=1
            
    #EXECUTE DATABASE QUERY TO UPDATE THE HOTEL AS FULLY SCRAPED
    try:
        update_hotel = ("UPDATE hotels SET hotel_scraped=1 WHERE id=%s")
        cursor.execute(update_hotel, (i[0],))
        cnxn.commit()
        print("HOTEL UPDATE SUCCESSFUL")
        print(i[2]+" SUCCESSFULLY SCRAPED")
    except Exception as e:
        print("SQL ERROR: COULD NOT UPDATE HOTEL")
        error_hotel_rows.append(e)
        print(e)
    